In [1]:
import os, sys
sys.path.append(os.path.abspath('..'))
from src.services.mail_client import send_email
from src.parsers.base_parser import BaseParser
from config.settings import DATA_DIR

In [23]:
file1 = BaseParser(os.path.join(DATA_DIR, 'PLF_Positions20260225.csv'))
df1 = file1.load_into_dataframe(header=0,index_col=False)

     FundID          ISIN      CUSIP Ticker    SEDOL  Basket Shares Currency  \
1158   HRQL  US8522341036  852234103    XYZ  BYNZGK1          87335      USD   
1159   HRA7  US8522341036  852234103    XYZ  BYNZGK1          53843      USD   
1160   HRTH  US98978V1035  98978V103    ZTS  B95WG16         520748      USD   

     Country Restriction Trade Country  IDENTIFIER  ...  FUND NAME  \
1158                   N            US         NaN  ...        NaN   
1159                   N            US         NaN  ...        NaN   
1160                   N            US         NaN  ...        NaN   

      BASKET DATE  CORPORATE ACTION FACTOR  INTEREST FACTOR  SUPPRESSPRICE  \
1158          NaN                      NaN              NaN            NaN   
1159          NaN                      NaN              NaN            NaN   
1160          NaN                      NaN              NaN            NaN   

      SUPPLEMENTAL_ID_1  SUPPLEMENTAL_ID_2  RIC  BBG  DERIV_TYPE  
1158       ST.US.U

In [54]:
import pandas as pd
df2 = df1.copy()
df2['FundID'] = df2['FundID'].astype(str).str.split('-').str[0].str.strip()
df_full = df2.drop_duplicates(subset=['FundID', 'SUPPLEMENTAL_ID_1'], keep='first').drop(columns=['Basket Shares'])

df_subset = df2.groupby(['FundID','SUPPLEMENTAL_ID_1']).agg({'Basket Shares':'sum'}).drop_duplicates()

final_df = pd.merge(df_full, df_subset, on=['FundID', 'SUPPLEMENTAL_ID_1'], how='left')
print(final_df.head())
final_df.to_csv(os.path.join(DATA_DIR, 'PLF_Positions20260225_cleaned.csv'), index=False)

  FundID ISIN CUSIP Ticker SEDOL Currency Country Restriction Trade Country  \
0   HRQ3  NaN   NaN   1AEM   NaN      CAD                   N            CA   
1   HRQY  NaN   NaN   1BCE   NaN      CAD                   N            CA   
2   HRCJ  NaN   NaN   1BMO   NaN      CAD                   N            CA   
3   HRCJ  NaN   NaN   1BMO   NaN      CAD                   N            CA   
4   HRCJ  NaN   NaN   1BNS   NaN      CAD                   N            CA   

   IDENTIFIER  CIN  ...  BASKET DATE  CORPORATE ACTION FACTOR  \
0         NaN  NaN  ...          NaN                      NaN   
1         NaN  NaN  ...          NaN                      NaN   
2         NaN  NaN  ...          NaN                      NaN   
3         NaN  NaN  ...          NaN                      NaN   
4         NaN  NaN  ...          NaN                      NaN   

   INTEREST FACTOR  SUPPRESSPRICE             SUPPLEMENTAL_ID_1  \
0              NaN            NaN  EO.CA.TO.1AEM..C275.20260306   


In [ ]:
import pandas as pd
import os

df2 = df1.copy().reset_index(drop=True)

# Normalize FundID aliases (e.g., HRCL-TD -> HRCL)
df2["FundID_root"] = (
    df2["FundID"]
    .astype(str)
    .str.strip()
    .str.upper()
    .str.split("-", n=1)
    .str[0]
)

# Ensure numeric shares before aggregation
df2["Basket Shares"] = pd.to_numeric(df2["Basket Shares"], errors="coerce")

# Group key for consolidation (fund alias + instrument identity)
keys = ["FundID_root", "SUPPLEMENTAL_ID_1", "Ticker", "RIC"]

# Aggregate shares within each consolidated key
agg = (
    df2.groupby(keys, dropna=False)["Basket Shares"]
       .sum(min_count=1)
       .rename("Basket Shares_agg")
       .reset_index()
)

# Keep one representative row per consolidated key
base = df2.drop_duplicates(subset=keys, keep="first")

# Attach aggregated shares and overwrite FundID with consolidated root
final_df = base.merge(agg, on=keys, how="left")
final_df["FundID"] = final_df["FundID_root"]
final_df["Basket Shares"] = final_df["Basket Shares_agg"]

# Cleanup helper columns and preserve original column order
final_df = final_df.drop(columns=["FundID_root", "Basket Shares_agg"])
final_df = final_df[df1.columns]

print(final_df.head())
print(f"Input rows: {len(df1):,} | Output rows: {len(final_df):,}")

final_df.to_csv(
    os.path.join(DATA_DIR, "PLF_Positions20260225_cleaned.csv"),
    index=False
)